In [ ]:
# Import necessary modules
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice

In [ ]:
# Set the path of the Database files
oPath = 'database/tables'
# Read all the sheets from the excel file.
students = pd.read_csv('{}/students.csv'.format(oPath), header = 0)
classes = pd.read_csv('{}/classes.csv'.format(oPath), header = 0)
tests = pd.read_csv('{}/tests.csv'.format(oPath), header = 0)
test_results = pd.read_csv('{}/test_results.csv'.format(oPath), header = 0)
attendance = pd.read_csv('{}/attendance.csv'.format(oPath), header = 0)
hw = pd.read_csv('{}/hw_assignments.csv'.format(oPath), header = 0)
hw_results = pd.read_csv('{}/hw_grades.csv'.format(oPath), header = 0)
participation = pd.read_csv('{}/participation.csv'.format(oPath), header = 0)
points = pd.read_csv('{}/points.csv'.format(oPath), header = 0)
# Merge attendance and class ID from students
attendance = pd.merge(attendance, students[['STUDENT_ID', 'CLASS_ID']], how = 'inner', on = 'STUDENT_ID')
# Get a list of ALL Class IDs
lClasses = classes['CLASS_ID'].to_list()
# Get a list of ALL Student IDs
lStudents = students['STUDENT_ID'].to_list()

In [ ]:
# FUNCTIONS
# Define a function to show values at scatter plot points
def fShowValuesScatter(axis, x, y, decimals, ychange):
    for i, j in zip(x, y):
        axis.annotate('{:.{}f}'.format(j, decimals), xy = (i, j), xytext=(ychange, 0), textcoords="offset points", va='center')
        
# Define a function to show values on bar plots
def fShowValuesOnBars(axs, decimals):
    def _show_on_single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height() + 0.05
            value = '{:.{}f}'.format(p.get_height(), decimals)
            ax.text(_x, _y, value, ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

# Define a function to show values on bar plots
def fShowValuesOnBarsCenter(axs, decimals, yMod):
    def _show_on_single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() / 2
            _y = p.get_y() + p.get_height() / yMod
            value = '{:.{}f}'.format(p.get_height(), decimals)
            ax.text(_x, _y, value, ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

## For All Classes

### *Prepare DataFrames*

In [ ]:
 # Define the dictionaries for each DF
dfAllStudents = {}
dfAllTestScores = {}
dfAllAttendance = {}
dfAllHWScores = {}
dfAllParticipation = {}
lAllTestIDs = {}
lAllHWIDs = {}
dfAllTests = {}
dfAllHW = {}
# Loop through all the class IDs to make a DF for each one
for i in lClasses:
    # Students DF
    dfAllStudents[i] = students.where(students['CLASS_ID'] == i)
    dfAllStudents[i].dropna(how = 'all', inplace = True)
    dfAllStudents[i] = dfAllStudents[i].astype({'STUDENT_ID': 'int32', 'STUDENT_AGE': 'int32', 'CLASS_ID': 'int32'})
    dfAllStudents[i] = dfAllStudents[i].reset_index(drop = True)
    # Test Scores DF
    dfAllTestScores[i] = test_results.where(test_results['CLASS_ID'] == i)
    dfAllTestScores[i].dropna(how = 'all', inplace = True)
    dfAllTestScores[i] = dfAllTestScores[i].astype({'STUDENT_ID': 'int32', 'TEST_ID': 'int32', 'TEST_SCORE': 'int32', 'TEST_SCORE_MAX': 'int32', 'CLASS_ID': 'int32'})
    dfAllTestScores[i]['TEST_DATE'] = pd.to_datetime(dfAllTestScores[i]['TEST_DATE'])
    dfAllTestScores[i] = dfAllTestScores[i].reset_index(drop = True)
    # Get a list of all Test IDs
    lAllTestIDs[i] = dfTestScores['TEST_ID'].unique()
    # Attendance DF
    dfAllAttendance[i] = attendance.where(attendance['CLASS_ID'] == i)
    dfAllAttendance[i].dropna(how = 'all', inplace = True)
    dfAllAttendance[i] = dfAllAttendance[i].astype({'STUDENT_ID': 'int32', 'CLASS_ID': 'int32'})
    dfAllAttendance[i]['CLASS_DATE'] = pd.to_datetime(dfAllAttendance[i]['CLASS_DATE'])
    dfAllAttendance[i] = dfAllAttendance[i].reset_index(drop = True)
    # HW Scores DF
    dfAllHWScores[i] = hw_results.where(hw_results['CLASS_ID'] == i)
    dfAllHWScores[i].dropna(how = 'all', inplace = True)
    dfAllHWScores[i] = dfAllHWScores[i].astype({'STUDENT_ID': 'int32', 'HW_ID': 'int32', 'HW_SCORE': 'int32', 'HW_SCORE_MAX': 'int32', 'CLASS_ID': 'int32'})
    dfAllHWScores[i]['HW_ASSIGNED_DATE'] = pd.to_datetime(dfAllHWScores[i]['HW_ASSIGNED_DATE'])
    dfAllHWScores[i]['HW_TURNED_IN_DATE'] = pd.to_datetime(dfAllHWScores[i]['HW_TURNED_IN_DATE'])
    dfAllHWScores[i] = dfAllHWScores[i].reset_index(drop = True)
    # Get a list of all the HW IDs
    lAllHWIDs[i] = dfHWScores['HW_ID'].unique() 
    # Participation DF
    dfAllParticipation[i] = participation.where(participation['CLASS_ID'] == i)
    dfAllParticipation[i].dropna(how = 'all', inplace = True)
    dfAllParticipation[i] = dfAllParticipation[i].astype({'STUDENT_ID': 'int32', 'PARTICIPATION_ATTEMPTS': 'int32', 'PARTICIPATION_HINTS': 'int32', 'CLASS_ID': 'int32'})
    dfAllParticipation[i]['PARTICIPATION_DATETIME'] = pd.to_datetime(dfAllParticipation[i]['PARTICIPATION_DATETIME'])
    dfAllParticipation[i] = dfAllParticipation[i].reset_index(drop = True)
    # Tests DF
    dfAllTests[i] = tests[tests['TEST_ID'].isin(lTestIDs)]
    dfAllTests[i].dropna(how = 'all', inplace = True)
    dfAllTests[i]['TEST_DATE'] = pd.to_datetime(dfAllTests[i]['TEST_DATE'])
    dfAllTests[i] = dfAllTests[i].reset_index(drop = True)
    # HW DF
    dfAllHW[i] = hw[hw['HW_ID'].isin(lHWIDs)]
    dfAllHW[i].dropna(how = 'all', inplace = True)
    dfAllHW[i]['HW_ASSIGNED_DATE'] = pd.to_datetime(dfAllHW[i]['HW_ASSIGNED_DATE'])
    dfAllHW[i]['HW_DUE_DATE'] = pd.to_datetime(dfAllHW[i]['HW_DUE_DATE'])
    dfAllHW[i] = dfAllHW[i].reset_index(drop = True)

# Define the Dictionaries necessary for the date filter
dfAllTestScoresDate = {}
dfAllAttendanceDate = {}
dfAllHWScoresDate = {}
dfAllParticipationDate = {}
dfAllTestsDate = {}
dfAllHWDate = {}
# Filter by Year and Month
for i in lClasses:
    dfAllTestScoresDate[i] = dfAllTestScores[i].where((dfAllTestScores[i]['TEST_DATE'].dt.year == tiYear) & (dfAllTestScores[i]['TEST_DATE'].dt.month == tiMonth))
    dfAllAttendanceDate[i] = dfAllAttendance[i].where((dfAllAttendance[i]['CLASS_DATE'].dt.year == tiYear) & (dfAllAttendance[i]['CLASS_DATE'].dt.month == tiMonth))
    dfAllHWScoresDate[i] = dfAllHWScores[i].where((dfAllHWScores[i]['HW_ASSIGNED_DATE'].dt.year == tiYear) & (dfAllHWScores[i]['HW_ASSIGNED_DATE'].dt.month == tiMonth))
    dfAllParticipationDate[i] = dfAllParticipation[i].where((dfAllParticipation[i]['PARTICIPATION_DATETIME'].dt.year == tiYear) & (dfAllParticipation[i]['PARTICIPATION_DATETIME'].dt.month == tiMonth))
    dfAllTestsDate[i] = dfAllTests[i].where((dfAllTests[i]['TEST_DATE'].dt.year == tiYear) & (dfAllTests[i]['TEST_DATE'].dt.month == tiMonth))
    dfAllHWDate[i] = dfAllHW[i].where((dfAllHW[i]['HW_ASSIGNED_DATE'].dt.year == tiYear) & (dfAllHW[i]['HW_ASSIGNED_DATE'].dt.month == tiMonth))

# Can use this to loopify the above?
lDFsAll = [dfAllStudents, dfAllTestScores, dfAllAttendance, dfAllHWScores, dfAllParticipation, dfAllTests, dfAllHW]